### Visualize monthly steer prices using USDA NASS data

This notebook creates time series plots of monthly steer cattle prices, comparing nominal and real price trends from January 1953 to December 2024.

Click the badge below to open in Google Colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chuckgrigsby0/agec-370/blob/main/notebooks/02_plot_monthly_steer_prices.ipynb)


We first load the monthly cattle price data obtained from USDA NASS and the average BLS CPI-U indices.

In [1]:
import pandas as pd
import numpy as np

# Base URL for raw GitHub content
base_url = "https://raw.githubusercontent.com/chuckgrigsby0/agec-370/main/data/"

# Load monthly steer price data directly from GitHub URL
steer_prices_monthly = pd.read_csv(base_url + 'steer_prices_monthly_nass.csv')

# Load CPI data directly from GitHub URL
cpi = pd.read_csv(base_url + 'cpi_1913_2024.csv')

HTTPError: HTTP Error 404: Not Found

### Create datetime index for proper time series plotting

Monthly data requires a proper datetime index to enable chronological ordering and appropriate x-axis formatting in plots.

In [ ]:
# Create a datetime column by combining year and month_id
# month_id is a two-digit string ('01' for January, '12' for December)
# We create date strings in 'YYYY-MM-DD' format, using day=01 for first of month
steer_prices_monthly['date'] = pd.to_datetime(
    steer_prices_monthly['year'].astype(str) + '-' + 
    steer_prices_monthly['month_id'].astype(str) + '-01'
)

# Sort by date to ensure chronological order
# This is important for time series plotting and ensures data flows correctly
steer_prices_monthly = steer_prices_monthly.sort_values('date').reset_index(drop=True)

# Display the first few rows to verify datetime creation
print("First 5 observations:")
print(steer_prices_monthly[['year', 'month', 'month_id', 'date', 'nominal_price']].head())
print("\nLast 5 observations:")
print(steer_prices_monthly[['year', 'month', 'month_id', 'date', 'nominal_price']].tail())


### Merge CPI data with steer price data to compute real prices


Merge monthly cattle price data with annual CPI data on year. Note that CPI data is available at the annual level, so the same CPI value applies to all 12 months within each year. This combines the price data with the Consumer Price Index for inflation adjustment. `'left'` join keeps all cattle price observations, matching CPI where available.

In [ ]:
steer_prices_monthly = steer_prices_monthly.merge(
    cpi,  # CPI data loaded earlier
    left_on='year',  # Match on 'year' column in cattle data
    right_on='year',  # Match on 'year' column in CPI data
    how='left'  # Keep all cattle price records
)

Get the CPI value for the base year (2024). This will be used as the denominator to adjust all prices to 2024 dollars. 

In [ ]:
cpi_base_year = cpi[cpi['year'] == 2024]['annual_avg_cpi'].values[0]  # Should be 313.689
print(f"Base year (2024) CPI: {cpi_base_year}")

Calculate real (inflation-adjusted) prices in 2024 dollars. 

- Formula: Real Price = (Nominal Price / CPI in year t) × CPI in base year
- This adjusts all historical prices to be comparable in 2024 purchasing power

In [ ]:
steer_prices_monthly['real_price'] = (
    steer_prices_monthly['nominal_price'] / steer_prices_monthly['annual_avg_cpi']
) * cpi_base_year

# Display summary statistics for both nominal and real prices
print("\nSummary Statistics:")
print(steer_prices_monthly[['nominal_price', 'real_price']].describe())

### Create Visualizations

In [ ]:
# Import visualization libraries
import matplotlib.pyplot as plt  # For figure creation and aesthetic enhancements
import seaborn as sns  # For statistical plots with regression lines

# Set the style for seaborn plots
# 'whitegrid' provides a clean background with subtle gridlines
sns.set_style('whitegrid')

### Line Plots

In the following, we will create separate line plots of nominal and real monthly steer prices. First, we create the nominal monthly price plot.

In [ ]:
# Create a larger figure for better visibility of monthly data
# figsize=(14, 6) provides a wide plot to accommodate 864 monthly observations
plt.figure(figsize=(14, 6))

sns.lineplot(
    data=steer_prices_monthly,  # Use the merged dataframe
    x='date',  # Date on x-axis (datetime format)
    y='nominal_price',  # Nominal price on y-axis
    linewidth=1.0,  # Thinner line due to high data density
    color='darkblue'  # Consistent color scheme
)

# Add titles and axis labels
plt.title("Nominal Steer Prices (Monthly)", fontsize=14, weight='bold')  # Plot title
plt.xlabel("Year", fontsize=12, weight='bold')  # X-axis title
plt.ylabel("Price ($/CWT)", fontsize=12, weight='bold')  # Y-axis title

# Improve x-axis readability for long time series
# Rotate labels for better fit
plt.xticks(rotation=90, ha='center')  # 90-degree rotation, centered

# Adjust layout to prevent label cutoff
plt.tight_layout()

plt.show()

Next, we'll create the same figure but use real steer prices.

In [ ]:
# Create a larger figure for better visibility of monthly data
# figsize=(14, 6) provides a wide plot to accommodate 864 monthly observations
plt.figure(figsize=(14, 6))

sns.lineplot(
    data=steer_prices_monthly,  # Use the merged dataframe
    x='date',  # Date on x-axis (datetime format)
    y='real_price',  # Real price on y-axis
    linewidth=1.0,  # Thinner line due to high data density
    color='darkred'  # Red color for real prices
)

# Add titles and axis labels
plt.title("Real Steer Prices (Monthly, 2024 Dollars)", fontsize=14, weight='bold')  # Plot title
plt.xlabel("Year", fontsize=12, weight='bold')  # X-axis title
plt.ylabel("Price ($/CWT)", fontsize=12, weight='bold')  # Y-axis title

# Improve x-axis readability for long time series
# Rotate labels for better fit
plt.xticks(rotation=90, ha='center')  # 90-degree rotation, centered

# Adjust layout to prevent label cutoff
plt.tight_layout()

plt.show()

### Analyzing seasonal trends

We can analyze seasonal trends by averaging prices for each month during a specific time period. 

In [ ]:
# Obtain data for years 2000 to 2024
steer_prices_monthly_2000_2024 = steer_prices_monthly[steer_prices_monthly['year'].between(2000, 2024)]

# Average nominal price from 2000 to 2024
avg_nominal_2000_2024 = steer_prices_monthly_2000_2024.groupby('month_id')['nominal_price'].mean().reset_index()

# Average real price from 2000 to 2024
avg_real_2000_2024 = steer_prices_monthly_2000_2024.groupby('month_id')['real_price'].mean().reset_index()

The next code block obtains the month labels for each month ID. 

In [ ]:
unique_months =steer_prices_monthly[['month', 'month_id']].drop_duplicates().reset_index(drop=True)

avg_nominal_2000_2024 = avg_nominal_2000_2024.merge(
    unique_months,
    on='month_id',
    how='left'
)
avg_real_2000_2024 = avg_real_2000_2024.merge(
    unique_months,
    on='month_id',
    how='left'
)

### Average Nominal Prices by Month (2000-2024)

We first plot average nominal prices for each month.

In [ ]:
# Create a larger figure for better visibility of monthly data
# figsize=(14, 6) provides a wide plot to accommodate 864 monthly observations
plt.figure(figsize=(14, 6))

sns.lineplot(
    data=avg_nominal_2000_2024,  # Use the merged dataframe
    x='month',  # Date on x-axis (datetime format)
    y='nominal_price',  # Nominal price on y-axis
    linewidth=1.0,  # Thinner line due to high data density
    color='darkblue'  # Consistent color scheme
)

# Add titles and axis labels
plt.title("Nominal Steer Prices (Monthly, 2000-2024)", fontsize=14, weight='bold')  # Plot title
plt.xlabel("Month", fontsize=12, weight='bold')  # X-axis title
plt.ylabel("Price ($/CWT)", fontsize=12, weight='bold')  # Y-axis title

# Improve x-axis readability for long time series
# Rotate labels for better fit
plt.xticks(rotation=0, ha='center')  # 90-degree rotation, centered

# Adjust layout to prevent label cutoff
plt.tight_layout()

plt.show()

### Average Real Prices by Month (2000-2024)

We next plot average real prices for each month. 

In [ ]:
# Create a larger figure for better visibility of monthly data
# figsize=(14, 6) provides a wide plot to accommodate 864 monthly observations
plt.figure(figsize=(14, 6))

sns.lineplot(
    data=avg_real_2000_2024,  # Use the merged dataframe
    x='month',  # Date on x-axis (datetime format)
    y='real_price',  # Real price on y-axis
    linewidth=1.0,  # Thinner line due to high data density
    color='darkorange'  # Consistent color scheme
)

# Add titles and axis labels
plt.title("Real Steer Prices (Monthly, 2000-2024)", fontsize=14, weight='bold')  # Plot title
plt.xlabel("Month", fontsize=12, weight='bold')  # X-axis title
plt.ylabel("Price ($/CWT)", fontsize=12, weight='bold')  # Y-axis title

# Improve x-axis readability for long time series
# Rotate labels for better fit
plt.xticks(rotation=0, ha='center')  # 90-degree rotation, centered

# Adjust layout to prevent label cutoff
plt.tight_layout()

plt.show()

### Advanced: Scatterplot with regression lines w/ Side-by-Side Comparison

We can also plot nominal and real steer prices side-by-side, to facilitate comparison. In addition, we include a regression line showing the long-run price trends.

In [ ]:
# Create figure with two subplots side by side
# figsize=(16, 8) creates a wide figure to accommodate two plots
# ncols=2 creates two columns for side-by-side comparison
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

# ============================================================
# LEFT PLOT: NOMINAL PRICES
# ============================================================

# For regression, we need numeric x-values
# Convert datetime to numeric (days since first observation)
steer_prices_monthly['date_numeric'] = (
    steer_prices_monthly['date'] - steer_prices_monthly['date'].min()
).dt.days

# Plot nominal prices with regression line
# regplot creates both scatterplot and fitted regression line
# ci=None turns off the confidence interval shading
sns.regplot(
    data=steer_prices_monthly,  # Use the merged dataframe
    x='date_numeric',  # Numeric date on x-axis for regression
    y='nominal_price',  # Nominal price on y-axis
    scatter_kws={
        'alpha': 0.3,  # More transparency due to high density of points
        's': 10,  # Smaller points due to 864 observations
        'color': 'darkblue',  # Color for nominal price points
        'edgecolors': 'none',  # No borders to reduce visual clutter
        'zorder': 3  # zorder=3 ensures points appear above lines
    },
    line_kws={
        'color': 'blue',  # Blue regression line for nominal prices
        'linewidth': 2.5,  # Thick line for visibility
        'linestyle': '--',  # Dashed line to distinguish from connected line
        'alpha': 0.7,  # Slight transparency
        'zorder': 2  # zorder=2 places regression line above grid but below points
    },
    ci=None,  # Turn off confidence interval shading
    ax=axes[0]  # Plot on first (left) subplot
)

# Add connected line plot to show month-to-month changes
# This connects consecutive months with a line to show temporal progression
axes[0].plot(
    steer_prices_monthly['date_numeric'],  # X values: numeric dates
    steer_prices_monthly['nominal_price'],  # Y values: nominal prices
    color='darkblue',  # Match scatter point color
    linewidth=0.8,  # Very thin line due to high data density
    alpha=0.4,  # More transparent to show as background connection
    zorder=1  # zorder=1 places connected line behind everything else
)

# Set title for nominal price subplot
# fontsize=14 makes subtitle prominent, weight='bold' emphasizes it
axes[0].set_title(
    'Nominal Steer Prices (Monthly)',
    fontsize=14,
    weight='bold',
    pad=15  # Add 15 points of padding above title
)

# Set x-axis label for nominal price subplot
axes[0].set_xlabel('Year', fontsize=12, weight='bold')

# Set y-axis label for nominal price subplot
axes[0].set_ylabel('Price ($ / CWT)', fontsize=12, weight='bold')

# Convert x-axis back to datetime labels for readability
# Get positions for year labels (every 5 years for readability)
years_to_show = range(1955, 2025, 5)  # Show every 5 years from 1955 to 2020
tick_positions = []
tick_labels = []

for year in years_to_show:
    # Find the first observation for this year
    year_data = steer_prices_monthly[steer_prices_monthly['year'] == year]
    if not year_data.empty:
        tick_positions.append(year_data['date_numeric'].iloc[0])
        tick_labels.append(str(year))

axes[0].set_xticks(tick_positions)
axes[0].set_xticklabels(tick_labels)

# Increase font size of tick labels and rotate x-axis labels to 45 degrees
# labelsize=11 makes year and price values easier to read
# rotation=45 for x-axis makes diagonal year labels that don't overlap
axes[0].tick_params(axis='y', labelsize=11)  # Y-axis tick label size
axes[0].tick_params(axis='x', labelsize=11, rotation=45)  # X-axis: larger labels, 45° rotation

# Add grid for easier reading of values
axes[0].grid(
    axis='both',  # Add gridlines for both x and y axes
    alpha=0.3,  # Make gridlines subtle (30% opacity)
    linestyle='-',  # Use solid lines
    linewidth=0.5  # Make gridlines thin
)

# ============================================================
# RIGHT PLOT: REAL PRICES (2024 DOLLARS)
# ============================================================

# Plot real prices with regression line
# Same structure as nominal plot but for inflation-adjusted prices
sns.regplot(
    data=steer_prices_monthly,  # Use the same dataframe
    x='date_numeric',  # Numeric date on x-axis for regression
    y='real_price',  # Real (inflation-adjusted) price on y-axis
    scatter_kws={
        'alpha': 0.3,  # More transparency due to high density of points
        's': 10,  # Smaller points due to 864 observations
        'color': 'darkred',  # Red color for real price points
        'edgecolors': 'none',  # No borders to reduce visual clutter
        'zorder': 3  # Points appear above lines
    },
    line_kws={
        'color': 'red',  # Red regression line for real prices
        'linewidth': 2.5,  # Thick line for visibility
        'linestyle': '--',  # Dashed line style
        'alpha': 0.7,  # Slight transparency
        'zorder': 2  # Regression line above grid but below points
    },
    ci=None,  # Turn off confidence interval shading
    ax=axes[1]  # Plot on second (right) subplot
)

# Add connected line plot to show month-to-month changes
# Connects consecutive months to show temporal progression for real prices
axes[1].plot(
    steer_prices_monthly['date_numeric'],  # X values: numeric dates
    steer_prices_monthly['real_price'],  # Y values: real prices
    color='darkred',  # Match scatter point color
    linewidth=0.8,  # Very thin line due to high data density
    alpha=0.4,  # More transparent to show as background connection
    zorder=1  # Connected line behind everything else
)

# Set title for real price subplot
axes[1].set_title(
    'Real Steer Prices (Monthly, 2024 Dollars)',
    fontsize=14,
    weight='bold',
    pad=15  # Add padding above title
)

# Set x-axis label for real price subplot
axes[1].set_xlabel('Year', fontsize=12, weight='bold')

# Set y-axis label for real price subplot
axes[1].set_ylabel('Price ($ / CWT)', fontsize=12, weight='bold')

# Convert x-axis back to datetime labels for readability
# Use the same tick positions and labels as the left plot for consistency
axes[1].set_xticks(tick_positions)
axes[1].set_xticklabels(tick_labels)

# Increase font size of tick labels (numbers on axes)
# labelsize=11 makes year and price values easier to read
axes[1].tick_params(axis='y', labelsize=11)  # Y-axis tick label size
axes[1].tick_params(axis='x', labelsize=11, rotation=45)  # X-axis: larger labels, 45° rotation

# Add grid for easier reading of values
axes[1].grid(
    axis='both',  # Add gridlines for both x and y axes
    alpha=0.3,  # Make gridlines subtle (30% opacity)
    linestyle='-',  # Use solid lines
    linewidth=0.5  # Make gridlines thin
)

# Add main title for entire figure
# This overarching title appears above both subplots
fig.suptitle(
    'Cattle Steer Prices Over Time (Monthly): Nominal vs Real Comparison',
    fontsize=16,
    weight='bold',
    y=0.98  # Position slightly above subplots (adjusted for taller figure)
)

# Adjust layout to prevent label cutoff and ensure proper spacing
# tight_layout() automatically adjusts subplot parameters for clean appearance
# rect parameter reserves space: [left, bottom, right, top] in figure coordinates
# bottom=0.05 provides extra space at bottom for rotated year labels
plt.tight_layout(rect=[0, 0.05, 1, 0.96])

plt.show()